In [2]:
import pandas as pd
import time
from datetime import datetime, timedelta
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


def initialize(url, load_sleep=10):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    time.sleep(load_sleep)  # to let the HTML load
    return driver

In [3]:
url = 'https://www.sofascore.com/aston-villa-leicester-city/GP'
url = 'https://www.sofascore.com/genclerbirligi-antalyaspor/glbscgd'
driver = initialize(url)

Trying to download new driver from http://chromedriver.storage.googleapis.com/80.0.3987.106/chromedriver_mac64.zip
Unpack archive /Users/TysonWu/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver.zip


0. Run schedule_crawler.py to get event_id, hkjc_url, est. of game_starting_time -> schedule.csv
0. Manual input sofascore_url in schedule.csv 
0. Schedule run of this script (ie. * * * * * python sofascore_timer.py --event_id <event_id>)
1. Auto crontab call this script with event_id as arg
2. This script finds hkjc_url from event_id from schedule.csv
3. This script calls crawler.py with hkjc_url as arg
4. While crawler crawls, this script loop through time from sofascore_url
5. When this script detects a reach of a minute (eg. 70') then enable signal check
6. Crawler.py ends when hkjc corner not available; This script ends when 'Ended' is shown in timer
7. For analysis, merge the sofascore_timer.csv with hkjc data to get accurate timepoints

Problem:
- Will this script terminate if it is a game with no corner data?

In [32]:
event_id = '20200309MON11'
row = pd.DataFrame({'event_id': event_id, 
                    'game_start': None,
                    'ht_start': None,
                    'ht_end': None,
                    'game_end': None},index=[0])
start_trigger = False
ht_trigger = False

while(True):
    timer = driver.find_element_by_class_name('js-details-component-startTime-container')
    time_text = timer.text.split("\n")[-1]
    print(time_text)
    if (time_text.startswith('0')) & (start_trigger==False) :
        start_trigger = True
        row['game_start'] = datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S")
    if ('Halftime' in time_text) & (ht_trigger==False):
        ht_trigger = True
        row['ht_start'] = datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S")
    if ('Halftime' not in time_text) & (ht_trigger==True):
        row['ht_end'] = datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S")
        
    # if reach a certain time, enable signal check

    if 'End' in time_text:
        row['game_end'] = datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S")
        break
    time.sleep(5)
row.to_csv('sofascore_timer.csv', index=False, mode='a')

Ended


## Find other statistics

In [113]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


def initialize(url, load_sleep=10):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    time.sleep(load_sleep)  # to let the HTML load
    return driver


def clean(s):
    s = re.sub('\n','',s)
    return s


def reshaping(home, i, away): # list of 3: [stat, incident, stat]
    i = i.lower()
    return {i+'_h': home, i+'_a': away}


def scrap_stat(driver):
    stat_all = driver.find_element_by_id('statistics-period-ALL')
    soup = BeautifulSoup(stat_all.get_attribute('innerHTML'),'html.parser')
    content = [clean(x).strip() for x in soup.find_all(text=True) if x != '\n']
    iterator = iter(content)
    df_dict = {'timestamp': datetime.strftime(datetime.now(),'%Y-%m-%d %H:%M:%S')}
    for i in iterator:
        df_dict.update(reshaping(i, next(iterator), next(iterator)))
    df = pd.DataFrame(df_dict, index=[0])
    return df


url = 'https://www.sofascore.com/aston-villa-leicester-city/GP'
driver = initialize(url)
while(True):
    scrap_stat(driver)


Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/TysonWu/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]


,timestamp,ball possession_h,ball possession_a,total shots_h,total shots_a,shots on target_h,shots on target_a,shots off target_h,shots off target_a,blocked shots_h,...,duels won_h,duels won_a,aerials won_h,aerials won_a,tackles_h,tackles_a,interceptions_h,interceptions_a,clearances_h,clearances_a
0,2020-03-11 01:33:56,64%,36%,15,4,7,1,7,3,1,...,62,58,18,15,24,23,8,10,12,22
